In [186]:
import pandas
import numpy
import pygsheets
import datetime
import pytz
import re

In [189]:
# define the excel file to save tables in
current_time = datetime.datetime.now(pytz.timezone('EST')).strftime("%Y-%m-%d_T%H%M%S")
excel_writer = pandas.ExcelWriter('AsiaGasTracker-Terminals-SummarySheets-'+current_time+'.xlsx')

In [190]:
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
#spreadsheet = gc.open_by_key('1tcS6Wd-Wp-LTDpLzFgJY_RSNDnbyubW3J_9HKIAys4A')
#spreadsheet = gc.open_by_key('1MrghwBeCz8Tzgua7CWGg_KoXKVZsV7r0kHMYHYqnNTg') # July 2022 terminals version
spreadsheet = gc.open_by_key('1FVuw76YU-jmAzfSWJ2ixwBQB_wXEK9aGCa4Q6OCOUzY') # July 2022 with May 2023 updates for Asia gas tracker

terms_df_orig = spreadsheet.worksheet('title', 'Terminals').get_as_df(start='A2')

# drop any empty/imcomplete rows (those without a wiki page)
terms_df_orig = terms_df_orig.loc[terms_df_orig.Wiki != '']

#get other relevant sheets
owners_df_orig = spreadsheet.worksheet('title', 'Terminal operators/owners (1/3)').get_as_df(start='A2')
owners_df_orig.set_index('ComboID', inplace=True)

parent_metadata_df = spreadsheet.worksheet('title', 'Parent metadata (3/3)').get_as_df(start='A2')
parent_metadata_df.set_index('Parent', inplace=True)

In [191]:
terms_df_orig.replace('--', numpy.nan, inplace=True)

In [192]:
region_df_orig = spreadsheet.worksheet('title', 'Country dictionary').get_as_df(start='A2')
region_df_agt = region_df_orig.loc[region_df_orig['AsiaGasTracker']=='Yes']

region_df_touse = region_df_agt.copy()

### create specific dataframes for region, country_ratios, owners_df

In [193]:
owners_df_touse = owners_df_orig.loc[owners_df_orig['Country'].isin(region_df_touse['Country'].tolist())]
terms_df_touse = terms_df_orig.loc[terms_df_orig['Country'].isin(region_df_touse['Country'].tolist())]

In [194]:
status_list = ['Proposed', 
               'Construction', 
               'Shelved', 
               'Cancelled', 
               'Operating', 
               'Idle', 
               'Mothballed', 
               'Retired']
country_list = sorted(set(region_df_touse['Country'].tolist()))
region_list = sorted(set(region_df_touse['Region'].tolist()))

In [195]:
excel_status_list = ['Proposed', 
                     'Construction', 
                     'In Development (Proposed + Construction)', 
                     'Shelved', 
                     'Cancelled', 
                     'Operating', 
                     'Idle', 
                     'Mothballed', 
                     'Retired']

### no. export trains by country/region, project status

In [196]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['Import/Export']=='Export') & (terms_df_touse['Fuel']=='LNG')]

ntrains_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
ntrains_by_region = pandas.DataFrame(0, columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_country[status] = terms_df_subset_status.groupby('Country').size()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_region[status] = terms_df_subset_status.groupby('Region').size()

# fille NaN with 0.0
ntrains_by_region = ntrains_by_region.fillna(0)
ntrains_by_country = ntrains_by_country.fillna(0)

ntrains_by_region['In Development (Proposed + Construction)'] = ntrains_by_region[['Proposed','Construction']].sum(axis=1)
ntrains_by_country['In Development (Proposed + Construction)'] = ntrains_by_country[['Proposed','Construction']].sum(axis=1)

ntrains_by_country = ntrains_by_country[excel_status_list]
ntrains_by_region = ntrains_by_region[excel_status_list]

ntrains_by_region.index.name = 'Region'
ntrains_by_country.index.name = 'Country'

totals_row = ntrains_by_region.sum(axis=0)
totals_row.name = 'Total'
ntrains_by_region = ntrains_by_region.append(totals_row)

totals_row = ntrains_by_country.sum(axis=0)
totals_row.name = 'Total'
ntrains_by_country = ntrains_by_country.append(totals_row)

ntrains_by_region.replace(0,'',inplace=True)
ntrains_by_country.replace(0,'',inplace=True)

ntrains_by_region.to_excel(excel_writer, sheet_name='LNG export trains by region')#, index=False)
ntrains_by_country.to_excel(excel_writer, sheet_name='LNG export trains by country')#, index=False)

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/740818966.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ntrains_by_region = ntrains_by_region.append(totals_row)
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/740818966.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ntrains_by_country = ntrains_by_country.append(totals_row)


In [197]:
ntrains_by_region

,Proposed,Construction,In Development (Proposed + Construction),Shelved,Cancelled,Operating,Idle,Mothballed,Retired
Region,,,,,,,,,
Asia,2.0,5.0,7.0,,17.0,23.0,,,10.0
Total,2.0,5.0,7.0,,17.0,23.0,,,10.0


### no. import trains by country/region, project status

In [198]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['Import/Export']=='Import') & (terms_df_touse['Fuel']=='LNG')]

ntrains_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
ntrains_by_region = pandas.DataFrame(0, columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_country[status] = terms_df_subset_status.groupby('Country').size()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    ntrains_by_region[status] = terms_df_subset_status.groupby('Region').size()

# fille NaN with 0.0
ntrains_by_region = ntrains_by_region.fillna(0)
ntrains_by_country = ntrains_by_country.fillna(0)

ntrains_by_region['In Development (Proposed + Construction)'] = ntrains_by_region[['Proposed','Construction']].sum(axis=1)
ntrains_by_country['In Development (Proposed + Construction)'] = ntrains_by_country[['Proposed','Construction']].sum(axis=1)

ntrains_by_country = ntrains_by_country[excel_status_list]
ntrains_by_region = ntrains_by_region[excel_status_list]

ntrains_by_region.index.name = 'Region'
ntrains_by_country.index.name = 'Country'

totals_row = ntrains_by_region.sum(axis=0)
totals_row.name = 'Total'
ntrains_by_region = ntrains_by_region.append(totals_row)

totals_row = ntrains_by_country.sum(axis=0)
totals_row.name = 'Total'
ntrains_by_country = ntrains_by_country.append(totals_row)

ntrains_by_region.replace(0,'',inplace=True)
ntrains_by_country.replace(0,'',inplace=True)

ntrains_by_region.to_excel(excel_writer, sheet_name='LNG import terminals by region')
ntrains_by_country.to_excel(excel_writer, sheet_name='LNG import terminals by country')


===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/2687660920.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ntrains_by_region = ntrains_by_region.append(totals_row)
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/2687660920.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ntrains_by_country = ntrains_by_country.append(totals_row)


In [199]:
ntrains_by_region

,Proposed,Construction,In Development (Proposed + Construction),Shelved,Cancelled,Operating,Idle,Mothballed,Retired
Region,,,,,,,,,
Asia,97.0,43.0,140.0,22.0,47.0,112.0,,1.0,
Total,97.0,43.0,140.0,22.0,47.0,112.0,,1.0,


### capacity (mtpa) of import trains by country/region, project status

In [200]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['Import/Export']=='Import') & (terms_df_touse['Fuel']=='LNG')]

cap_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
cap_by_region = pandas.DataFrame(0, columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_country[status] = terms_df_subset_status.groupby('Country')['CapacityInMtpa'].sum()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_region[status] = terms_df_subset_status.groupby('Region')['CapacityInMtpa'].sum()

#fille NaN with 0.0
cap_by_region = cap_by_region.fillna(0)
cap_by_country = cap_by_country.fillna(0)

cap_by_region['In Development (Proposed + Construction)'] = cap_by_region[['Proposed','Construction']].sum(axis=1)
cap_by_country['In Development (Proposed + Construction)'] = cap_by_country[['Proposed','Construction']].sum(axis=1)

cap_by_country = cap_by_country[excel_status_list]
cap_by_region = cap_by_region[excel_status_list]

cap_by_region.index.name = 'Region'
cap_by_country.index.name = 'Country'

totals_row = cap_by_region.sum(axis=0)
totals_row.name = 'Total'
cap_by_region = cap_by_region.append(totals_row)

totals_row = cap_by_country.sum(axis=0)
totals_row.name = 'Total'
cap_by_country = cap_by_country.append(totals_row)

cap_by_region.replace(0,'',inplace=True)
cap_by_country.replace(0,'',inplace=True)

cap_by_region.to_excel(excel_writer, 'LNG import capacity by region')
cap_by_country.to_excel(excel_writer, 'LNG import capacity by country')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/3084418678.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cap_by_region = cap_by_region.append(totals_row)
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/3084418678.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cap_by_country = cap_by_country.append(totals_row)


### capacity (mtpa) of export trains by country/region, project status

In [201]:
terms_df_subset = terms_df_touse.copy()[(terms_df_touse['Import/Export']=='Export') & (terms_df_touse['Fuel']=='LNG')]

cap_by_country = pandas.DataFrame(0, columns=status_list, index=country_list)
cap_by_region = pandas.DataFrame(0, columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_country[status] = terms_df_subset_status.groupby('Country')['CapacityInMtpa'].sum()

print('===country-level calculations===')
for status in status_list:
    print(status)
    terms_df_subset_status = terms_df_subset.copy()[terms_df_subset['Status']==status]
    cap_by_region[status] = terms_df_subset_status.groupby('Region')['CapacityInMtpa'].sum()

#fille NaN with 0.0
cap_by_region = cap_by_region.fillna(0)
cap_by_country = cap_by_country.fillna(0)

cap_by_region['In Development (Proposed + Construction)'] = cap_by_region[['Proposed','Construction']].sum(axis=1)
cap_by_country['In Development (Proposed + Construction)'] = cap_by_country[['Proposed','Construction']].sum(axis=1)

cap_by_country = cap_by_country[excel_status_list]
cap_by_region = cap_by_region[excel_status_list]

cap_by_region.index.name = 'Region'
cap_by_country.index.name = 'Country'

totals_row = cap_by_region.sum(axis=0)
totals_row.name = 'Total'
cap_by_region = cap_by_region.append(totals_row)

totals_row = cap_by_country.sum(axis=0)
totals_row.name = 'Total'
cap_by_country = cap_by_country.append(totals_row)

cap_by_region.replace(0,'',inplace=True)
cap_by_country.replace(0,'',inplace=True)

cap_by_region.to_excel(excel_writer, 'LNG export capacity by region')
cap_by_country.to_excel(excel_writer, 'LNG export capacity by country')

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/2541422666.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cap_by_region = cap_by_region.append(totals_row)
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/2541422666.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cap_by_country = cap_by_country.append(totals_row)


## owner analysis

In [202]:
# this cell takes the terminals data set (call it "terms_df_all", here we're using the indev terminals subset)
# and loops throw every row, breaks up the parent and % ownership strings,
# and outputs a dataframe owner_parent_calculations_df that has a bunch of different useful info related
# to parent shares, capacity, etc.
owner_parent_calculations_df = pandas.DataFrame() # empty df for storing owner/parent stuff

for idx,row in terms_df_touse.iterrows():
    parent_string = row.Parent #.values[0]

    # the two lines below go through the parent entries in the sheet and split them,
    # recording the percents and parents in separate lists, to be dealt with later
    parent_list = re.sub(' \[.*?\]', '', parent_string).split('; ') # all entries must have an "Owner [%]" syntax, so [unknown %] is included
    percent_list = [float(i.rstrip('%'))/100. for i in re.findall('\\d+(?:\\.\\d+)?%', parent_string)]

    # if there are fewer recorded percents than number of parents
    if parent_list.__len__()!=percent_list.__len__():
        # if there are no percents, make them all equal
        if percent_list==[]:
            percent_list = [1/parent_list.__len__() for i in parent_list]
        # otherwise, get the difference and distribute the remainder equally among remaining parents
        else:
            nmissing = parent_list.__len__()-percent_list.__len__()
            # distribute nans evenly
            total = numpy.nansum(percent_list)
            leftover = 1-total
            percent_list += [leftover/nmissing]*nmissing

    # now go through each parent and associated percent ownership
    # and store it alongside the project's name along with capacity, status, region, etc.
    # this is stored inside the big dataframe owner_parent_calculations_df
    for p_idx,parent in enumerate(parent_list):
        if parent not in parent_metadata_df.index.to_list():
            parent_metadata_df.loc[parent,'ParentHQCountry']='Unknown'
        owner_parent_calculations_df = pandas.concat([owner_parent_calculations_df, 
                                                      pandas.DataFrame([{'Parent':parent, 'ComboID':row.ComboID,
                                                                         'FractionOwnership':percent_list[p_idx],
                                                                         'PercentString':'{:.2f}%'.format(percent_list[p_idx]*100),
                                                                         'ParentHQCountry':parent_metadata_df.loc[parent_metadata_df.index==parent]['ParentHQCountry'].values[0],
                                                                         'ParentHQRegion':parent_metadata_df.loc[parent_metadata_df.index==parent]['ParentHQRegion'].values[0],
                                                                         'TerminalCountry':row.Country,
                                                                         'TerminalRegion':row.Region,
                                                                         'Status':row.Status,
                                                                         'Import/Export':row['Import/Export'],
                                                                         'CapacityInMtpa':row.CapacityInMtpa,
                                                                         'ProjectWiki':row.Wiki,
                                                                         'ProjectName':row.TerminalName+' '+str(row.UnitName)}])])

owner_parent_calculations_df.reset_index(drop=True, inplace=True)
owner_parent_calculations_df['CapacityOwnedInMtpa'] = owner_parent_calculations_df.FractionOwnership*owner_parent_calculations_df.CapacityInMtpa
owner_parent_calculations_df['ParentCapacityString'] = owner_parent_calculations_df.Parent + ' (' + owner_parent_calculations_df.CapacityOwnedInMtpa.round(1).astype(str).replace('nan','unknown') + ' mtpa)'

### ntrains export by parent company, project status

In [203]:
owners_ntrains_by_status_df = \
    owner_parent_calculations_df.loc[owner_parent_calculations_df['Import/Export']=='Export'].groupby(
    ['Parent','ParentHQCountry','Status']).agg({'Status':['count']}).unstack().droplevel(axis=1, level=[0,1])

owners_ntrains_by_status_df = owners_ntrains_by_status_df.reindex(columns=status_list)
owners_ntrains_by_status_df = owners_ntrains_by_status_df.reset_index().set_index('Parent')
owners_ntrains_by_status_df.columns.name = None

owners_ntrains_by_status_df['In Development (Proposed + Construction)'] = owners_ntrains_by_status_df[['Proposed','Construction']].sum(axis=1)

owners_ntrains_by_status_df = owners_ntrains_by_status_df.rename(columns={'Parent':'Parent Company',
                                                                          'ParentHQCountry':'Country'})
# rearrange the order of the columns for output
owners_ntrains_by_status_df = owners_ntrains_by_status_df[excel_status_list_countries]

totals_row = owners_ntrains_by_status_df.sum(axis=0, min_count=0)
totals_row.name = 'Total'
owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)
owners_ntrains_by_status_df.loc['Total','Country'] = ''

owners_ntrains_by_status_df = owners_ntrains_by_status_df.replace(numpy.nan, '')
owners_ntrains_by_status_df = owners_ntrains_by_status_df.replace(0, '')

owners_ntrains_by_status_df.to_excel(excel_writer, 'LNG export trains by owner')

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/3071515540.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)


### ntrains import by parent company, project status

In [204]:
owners_ntrains_by_status_df = \
    owner_parent_calculations_df.loc[owner_parent_calculations_df['Import/Export']=='Import'].groupby(
    ['Parent','ParentHQCountry','Status']).agg({'Status':['count']}).unstack().droplevel(axis=1, level=[0,1])

owners_ntrains_by_status_df = owners_ntrains_by_status_df.reindex(columns=status_list)
owners_ntrains_by_status_df = owners_ntrains_by_status_df.reset_index().set_index('Parent')
owners_ntrains_by_status_df.columns.name = None

owners_ntrains_by_status_df['In Development (Proposed + Construction)'] = owners_ntrains_by_status_df[['Proposed','Construction']].sum(axis=1)

owners_ntrains_by_status_df = owners_ntrains_by_status_df.rename(columns={'Parent':'Parent Company',
                                                                          'ParentHQCountry':'Country'})
# rearrange the order of the columns for output
owners_ntrains_by_status_df = owners_ntrains_by_status_df[excel_status_list_countries]

totals_row = owners_ntrains_by_status_df.sum(axis=0, min_count=0)
totals_row.name = 'Total'
owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)
owners_ntrains_by_status_df.loc['Total','Country'] = ''

owners_ntrains_by_status_df = owners_ntrains_by_status_df.replace(numpy.nan, '')
owners_ntrains_by_status_df = owners_ntrains_by_status_df.replace(0, '')

owners_ntrains_by_status_df.to_excel(excel_writer, 'LNG import trains by owner')

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/2693392009.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)


### capacity export by parent company, project status

In [205]:
owners_capacity_by_status_df = \
    owner_parent_calculations_df.loc[owner_parent_calculations_df['Import/Export']=='Export'].groupby(
    ['Parent','ParentHQCountry','Status']).agg({'CapacityInMtpa':['sum']}).unstack().droplevel(axis=1, level=[0,1])

owners_capacity_by_status_df = owners_capacity_by_status_df.reindex(columns=status_list)
owners_capacity_by_status_df = owners_capacity_by_status_df.reset_index().set_index('Parent')
owners_capacity_by_status_df.columns.name = None

owners_capacity_by_status_df['In Development (Proposed + Construction)'] = owners_capacity_by_status_df[['Proposed','Construction']].sum(axis=1)

owners_capacity_by_status_df = owners_capacity_by_status_df.rename(columns={'Parent':'Parent Company',
                                                                          'ParentHQCountry':'Country'})
# rearrange the order of the columns for output
owners_capacity_by_status_df = owners_capacity_by_status_df[excel_status_list_countries]

totals_row = owners_capacity_by_status_df.sum(axis=0, min_count=0)
totals_row.name = 'Total'
owners_capacity_by_status_df = owners_capacity_by_status_df.append(totals_row)
owners_capacity_by_status_df.loc['Total','Country'] = ''

owners_capacity_by_status_df = owners_capacity_by_status_df.replace(numpy.nan, '')
owners_capacity_by_status_df = owners_capacity_by_status_df.replace(0, '')

owners_capacity_by_status_df.to_excel(excel_writer, 'LNG export capacity by owner')

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/1758428578.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  owners_capacity_by_status_df = owners_capacity_by_status_df.append(totals_row)


### capacity import by parent company, project status

In [206]:
owners_capacity_by_status_df = \
    owner_parent_calculations_df.loc[owner_parent_calculations_df['Import/Export']=='Import'].groupby(
    ['Parent','ParentHQCountry','Status']).agg({'CapacityInMtpa':['sum']}).unstack().droplevel(axis=1, level=[0,1])

owners_capacity_by_status_df = owners_capacity_by_status_df.reindex(columns=status_list)
owners_capacity_by_status_df = owners_capacity_by_status_df.reset_index().set_index('Parent')
owners_capacity_by_status_df.columns.name = None

owners_capacity_by_status_df['In Development (Proposed + Construction)'] = owners_capacity_by_status_df[['Proposed','Construction']].sum(axis=1)

owners_capacity_by_status_df = owners_capacity_by_status_df.rename(columns={'Parent':'Parent Company',
                                                                          'ParentHQCountry':'Country'})
# rearrange the order of the columns for output
owners_capacity_by_status_df = owners_capacity_by_status_df[excel_status_list_countries]

totals_row = owners_capacity_by_status_df.sum(axis=0, min_count=0)
totals_row.name = 'Total'
owners_capacity_by_status_df = owners_capacity_by_status_df.append(totals_row)
owners_capacity_by_status_df.loc['Total','Country'] = ''

owners_capacity_by_status_df = owners_capacity_by_status_df.replace(numpy.nan, '')
owners_capacity_by_status_df = owners_capacity_by_status_df.replace(0, '')

owners_capacity_by_status_df.to_excel(excel_writer, 'LNG import capacity by owner')

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/926954435.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  owners_capacity_by_status_df = owners_capacity_by_status_df.append(totals_row)


### no. of operating terminals by start year, facility type (1980–2021)

In [207]:
year_indices = list(range(1980,2023))

terms_by_start_year_df = pandas.DataFrame(0.0, index=year_indices, columns=['Import terminals', 'Export terminals'])

terms_started_df = terms_df_touse[(terms_df_touse['Status'].isin(['Operating'])) &
                              (terms_df_touse['Fuel']=='LNG') & (terms_df_touse['Import/Export']=='Import')]
terms_by_start_year_df['Import terminals'] = terms_started_df.groupby('StartYearEarliest').size()

terms_started_df = terms_df_touse[(terms_df_touse['Status'].isin(['Operating'])) &
                              (terms_df_touse['Fuel']=='LNG') & (terms_df_touse['Import/Export']=='Export')]
terms_by_start_year_df['Export terminals'] = terms_started_df.groupby('StartYearEarliest').size()

terms_by_start_year_df.index.name = 'Start year'
terms_by_start_year_df.replace(numpy.nan,'',inplace=True)

totals_row = terms_by_start_year_df.sum(axis=0)
totals_row.name = 'Total'
terms_by_start_year_df = terms_by_start_year_df.append(totals_row)

terms_by_start_year_df.to_excel(excel_writer, 'LNG terminals by start year')

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/3337449361.py:16: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  totals_row = terms_by_start_year_df.sum(axis=0)
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/3337449361.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  terms_by_start_year_df = terms_by_start_year_df.append(totals_row)


### capacity of operating terminals by start year, facility type (1980–2021)

In [208]:
year_indices = list(range(1980,2023))

capacity_by_start_year_df = pandas.DataFrame(0.0, index=year_indices, columns=['Import terminals', 'Export terminals'])

terms_started_df = terms_df_touse.loc[(terms_df_touse['Status'].isin(['Operating'])) &
                                      (terms_df_touse['Fuel']=='LNG') & 
                                      (terms_df_touse['Import/Export']=='Import')]
capacity_by_start_year_df['Import terminals'] = terms_started_df.groupby('StartYearEarliest')['CapacityInMtpa'].sum()

terms_started_df = terms_df_touse.loc[(terms_df_touse['Status'].isin(['Operating'])) &
                                      (terms_df_touse['Fuel']=='LNG') & 
                                      (terms_df_touse['Import/Export']=='Export')]
capacity_by_start_year_df['Export terminals'] = terms_started_df.groupby('StartYearEarliest')['CapacityInMtpa'].sum(min_count=0)

capacity_by_start_year_df.index.name = 'Start year'
capacity_by_start_year_df.replace(numpy.nan,'',inplace=True)

totals_row = capacity_by_start_year_df.sum(axis=0)
totals_row.name = 'Total'
capacity_by_start_year_df = capacity_by_start_year_df.append(totals_row)

capacity_by_start_year_df.to_excel(excel_writer, 'LNG capacity by start year')

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/106313456.py:18: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  totals_row = capacity_by_start_year_df.sum(axis=0)
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/106313456.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  capacity_by_start_year_df = capacity_by_start_year_df.append(totals_row)


# save excel file

In [209]:
excel_writer.save()

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_70506/2638382022.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excel_writer.save()
